## Dataset

In [0]:
import tensorflow as tf
import os

data_zip = tf.keras.utils.get_file('dataset.zip',
                                        extract=True,
                                        cache_subdir=os.path.abspath('.'),
                                        origin='https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip')


# Model

In [1]:
from model import CycleGAN

gan = CycleGAN(mode='train', base='unet')
gan.build()

E:\ANACODA\envs\cs231n\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import os
gan.compile(learning_rate=2e-4)
gan.train('../dataset/horse2zebra/trainA/*.jpg', '../dataset/horse2zebra/trainB/*.jpg', epochs=100, 
          steps_per_epoch=100, batch_size=1, image_save_path='../output/temp/', model_save_path='../models/temp/',
          save_image_every_step=10, save_model_every_epoch=1,
          show_image=True, load_model=False, model_load_path='../models/')

Training starts...
Tensor("arg0:0", shape=(), dtype=string)
(256, 256, 3)
Tensor("arg0:0", shape=(), dtype=string)
(256, 256, 3)
Epoch 0 starts...
[Epoch 1/100] [Batch 1/100] [D_A loss: 1.690780, acc:  31%] [D_B loss: 2.013488, acc:  30%] [G_A loss: 5.387044] [G_B loss: 8.734776] time: 165.89577174186707


KeyboardInterrupt: 